# 线性插值法

## 以文件1为例

In [1]:
import pandas as pd
import datetime
import numpy as np

# 读取文件1
data = pd.read_excel('file1.xlsx')
X = data['时间']
# 将时间列由str转换成datetime类型
time_list = [datetime.datetime.strptime(str(i), '%Y/%m/%d %H:%M:%S.%f.') for i in X]
data['时间'] = time_list
n = len(X)

In [46]:
# 时间间隔下标
interval = []
# 时间间隔
delt_times = []
for i in range(len(time_list)-1):
    delt_time = time_list[i+1] - time_list[i]
    if delt_time.total_seconds() > 1.0:
        delt_times.append(delt_time.total_seconds())
        interval.append(i)

In [98]:
# List转成numpy的array类型,方便使用Fancy Indexing
interval_np = np.array(interval)
delt_times_np = np.array(delt_times)

# 所有时间间隔大于120s的下标
interval_np = interval_np[delt_times_np>120.0]
interval_np

array([  4217,   5031,   5419,   5858,   6778,   6864,   7425,   9403,
         9530,  10904,  10991,  13513,  14997,  16800,  18932,  19431,
        21237,  22324,  22885,  23838,  25026,  26156,  26383,  26918,
        27113,  27239,  28218,  32641,  32679,  32724,  32801,  32963,
        33200,  34744,  37024,  38002,  38185,  38323,  38697,  38909,
        39253,  40053,  40906,  41295,  41353,  41459,  41619,  41810,
        41987,  42026,  42140,  42534,  42874,  42941,  43621,  43753,
        43886,  43955,  44320,  45303,  45717,  47391,  49661,  49932,
        49934,  50487,  52822,  52866,  54640,  55026,  55862,  56718,
        56885,  57120,  57679,  57801,  58508,  59958,  61832,  62146,
        62641,  63298,  63590,  64026,  64515,  64591,  64633,  65610,
        66460,  66847,  68804,  70127,  70152,  71255,  71507,  72544,
        73123,  73371,  79761,  80232,  84056,  84603,  85740,  87123,
        87930,  88412,  90802,  92115,  92945,  93616,  93716,  97225,
      

In [2]:
# 生成全序列时间样本，无间隔
helper = pd.DataFrame({'时间':pd.date_range(start=data['时间'][0], end=data['时间'][n-1], freq='S')})
# 与文件1合并,间隔内的其他列全为Nan值
data = pd.merge(data, helper, on='时间', how='outer').sort_values('时间')

In [44]:
# 对GPS车速列进行线性插值
data['GPS车速'] = data['GPS车速'].interpolate(method='time')
# 带着插入后的index 写入文件
data.to_csv('gole.csv',encoding='utf-8', index_label='index')

In [26]:
# 插值后数据条目
print(len(data))

518137


In [92]:
# 读取文件,第一列是index
gole = pd.read_csv('gole.csv')

## 实验效果

删除原id=1314~1315之间插入的数据

这是我做实验用的,运行过这部分代码后,在对整个文件进行数据删除就会报错

因为其中的1314~1315已经被删除了,不能重复删除

In [93]:
gole.loc[gole['index']==1314]

,index,时间,GPS车速,X轴加速度,Y轴加速度,Z轴加速度,经度,纬度,发动机转速,扭矩百分比,瞬时油耗,油门踏板开度,空燃比,发动机负荷百分比,进气流量
1314,1314,2017-12-18 14:04:07,60.9,0.0,-0.486,-0.918,119.40564,25.954413,1537.0,13.0,0.29,0.075,0.1463,17.0,3.07


In [94]:
gole.loc[gole['index']==1315]

,index,时间,GPS车速,X轴加速度,Y轴加速度,Z轴加速度,经度,纬度,发动机转速,扭矩百分比,瞬时油耗,油门踏板开度,空燃比,发动机负荷百分比,进气流量
1383,1315,2017-12-18 14:05:16,14.6,0.0,-0.36,-0.954,119.410956,25.952053,1487.0,34.0,0.88,0.11,0.1465,38.0,9.01


In [95]:
start = gole[gole['index']==1314].index[0] + 1
start

1315

In [96]:
end = gole[gole['index']==1315].index[0] + 1 
end

1384

In [82]:
gole.drop(labels=np.arange(1315,1384),axis=0, inplace=True)

In [83]:
gole[gole['index']==1314]

,index,时间,GPS车速,X轴加速度,Y轴加速度,Z轴加速度,经度,纬度,发动机转速,扭矩百分比,瞬时油耗,油门踏板开度,空燃比,发动机负荷百分比,进气流量
1314,1314,2017-12-18 14:04:07,60.9,0.0,-0.486,-0.918,119.40564,25.954413,1537.0,13.0,0.29,0.075,0.1463,17.0,3.07


In [84]:
gole[gole['index']==1315]

,index,时间,GPS车速,X轴加速度,Y轴加速度,Z轴加速度,经度,纬度,发动机转速,扭矩百分比,瞬时油耗,油门踏板开度,空燃比,发动机负荷百分比,进气流量


In [85]:
gole[1314:1384]

,index,时间,GPS车速,X轴加速度,Y轴加速度,Z轴加速度,经度,纬度,发动机转速,扭矩百分比,瞬时油耗,油门踏板开度,空燃比,发动机负荷百分比,进气流量
1314,1314,2017-12-18 14:04:07,60.9,0.000,-0.486,-0.918,119.405640,25.954413,1537.0,13.0,0.29,0.075,0.1463,17.0,3.07
1384,1316,2017-12-18 14:05:17,15.9,0.000,-0.288,-0.900,119.410995,25.952028,1787.0,51.0,1.46,0.145,0.1464,62.0,14.86
1385,1317,2017-12-18 14:05:18,19.8,-0.108,-0.180,-1.008,119.411036,25.951999,1875.0,50.0,1.49,0.145,0.1468,60.0,15.27
1386,1318,2017-12-18 14:05:19,23.8,0.000,-0.252,-0.918,119.411088,25.951965,1437.0,21.0,0.64,0.110,0.1463,18.0,6.66
1387,1319,2017-12-18 14:05:20,25.2,0.000,-0.324,-0.918,119.411146,25.951924,1750.0,51.0,1.39,0.145,0.1473,60.0,14.23
1388,1320,2017-12-18 14:05:21,27.9,0.072,-0.342,-0.936,119.411210,25.951886,1837.0,54.0,1.56,0.150,0.1471,64.0,15.84
1389,1321,2017-12-18 14:05:22,30.9,0.000,-0.324,-0.954,119.411278,25.951840,1337.0,16.0,0.29,0.010,0.1463,22.0,3.10
1390,1322,2017-12-18 14:05:23,32.2,0.000,-0.378,-0.882,119.411350,25.951790,1812.0,54.0,1.56,0.150,0.1470,63.0,15.86
1391,1323,2017-12-18 14:05:24,34.0,0.000,-0.324,-0.882,119.411425,25.951738,1925.0,54.0,1.61,0.150,0.1466,63.0,16.36
1392,1324,2017-12-18 14:05:25,36.8,-0.054,-0.306,-0.936,119.411506,25.951683,1962.0,53.0,1.61,0.145,0.1468,63.0,16.47


## 删除所有间断点>2min之间插入的数据

In [99]:
for i in interval_np:
    start = gole[gole['index']==i].index[0] + 1
    end = gole[gole['index']==i+1].index[0] + 1 
    gole.drop(labels=np.arange(start,end),axis=0, inplace=True)

In [100]:
# 删除后条数
len(gole)

208420

In [104]:
# 最终插入数据条数
len(gole) - len(X)

22695

In [106]:
# 删除索引列
gole.drop(['index'],axis=1)

,时间,GPS车速,X轴加速度,Y轴加速度,Z轴加速度,经度,纬度,发动机转速,扭矩百分比,瞬时油耗,油门踏板开度,空燃比,发动机负荷百分比,进气流量
0,2017-12-18 13:42:13,0.0,0.000,-0.396,-0.900,119.367781,25.992420,775.0,18.0,58.02,0.000,0.1465,22.0,2.30
1,2017-12-18 13:42:14,0.0,0.000,-0.378,-0.882,119.367783,25.992418,775.0,17.0,60.30,0.000,0.1465,21.0,2.39
2,2017-12-18 13:42:15,0.0,0.000,-0.396,-0.882,119.367785,25.992416,775.0,17.0,55.24,0.000,0.1464,22.0,2.19
3,2017-12-18 13:42:16,0.0,0.000,-0.378,-0.900,119.367785,25.992416,762.0,17.0,55.75,0.000,0.1471,21.0,2.21
4,2017-12-18 13:42:17,0.0,0.000,-0.396,-0.882,119.367785,25.992416,762.0,16.0,56.00,0.000,0.1471,21.0,2.22
5,2017-12-18 13:42:18,0.0,0.000,-0.378,-0.900,119.367785,25.992416,787.0,18.0,63.33,0.000,0.1471,22.0,2.51
6,2017-12-18 13:42:19,0.0,0.000,-0.342,-0.936,119.367785,25.992416,687.0,21.0,62.06,0.050,0.1471,26.0,2.46
7,2017-12-18 13:42:20,0.0,0.000,-0.324,-0.936,119.367785,25.992416,900.0,25.0,0.36,0.080,0.1470,31.0,3.79
8,2017-12-18 13:42:21,4.5,0.000,-0.324,-0.918,119.367791,25.992408,1025.0,25.0,0.44,0.080,0.1467,32.0,4.51
9,2017-12-18 13:42:22,6.9,0.000,-0.324,-0.936,119.367801,25.992398,1137.0,26.0,0.46,0.080,0.1466,32.0,4.70


In [108]:
# 保存最终文件
gole.to_csv('./handel/文件1.csv',encoding='utf-8', index=0)

# 处理文件2

In [ ]:
import pandas as pd
import datetime
import numpy as np

# 读取文件1
data = pd.read_excel('file2.xlsx')
X = data['时间']
# 将时间列由str转换成datetime类型
time_list = [datetime.datetime.strptime(str(i), '%Y/%m/%d %H:%M:%S.%f.') for i in X]
data['时间'] = time_list
n = len(X)

In [ ]:
# 时间间隔下标
interval = []
# 时间间隔
delt_times = []
for i in range(len(time_list)-1):
    delt_time = time_list[i+1] - time_list[i]
    if delt_time.total_seconds() > 1.0:
        delt_times.append(delt_time.total_seconds())
        interval.append(i)

In [ ]:
# List转成numpy的array类型,方便使用Fancy Indexing
interval_np = np.array(interval)
delt_times_np = np.array(delt_times)

# 所有时间间隔大于120s的下标
interval_np = interval_np[delt_times_np>120.0]

In [ ]:
# 生成全序列时间样本，无间隔
helper = pd.DataFrame({'时间':pd.date_range(start=data['时间'][0], end=data['时间'][n-1], freq='S')})
# 与文件1合并,间隔内的其他列全为Nan值
data = pd.merge(data, helper, on='时间', how='outer').sort_values('时间')

In [ ]:
# 对GPS车速列进行线性插值
data['GPS车速'] = data['GPS车速'].interpolate(method='time')
# 带着插入后的index 写入文件
data.to_csv('gole.csv',encoding='utf-8', index_label='index')

In [ ]:
# 读取文件,第一列是index
gole = pd.read_csv('gole.csv')

In [ ]:
for i in interval_np:
    start = gole[gole['index']==i].index[0] + 1
    end = gole[gole['index']==i+1].index[0] + 1 
    gole.drop(labels=np.arange(start,end),axis=0, inplace=True)

In [ ]:
# 删除索引列
gole.drop(['index'],axis=1)

In [ ]:
# 保存最终文件
gole.to_csv('./handel/文件2.csv',encoding='utf-8', index=0)